## 9	FLUJO DE CAJA LIBRE DESCONTADO: VALORACIÓN DE UNA EMPRESA CON PYTHON.

## 9.4.1	Estimación del crecimiento futuro de los ingresos

In [72]:
import requests
import numpy as np
import pandas as pd

In [73]:
company = 'GOOG'
demo = 'api key' #Aquí debe ingresar su clave
IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={demo}').json()
count = 0

In [74]:
#Obtener el crecimiento de los ingresos para estimar los ingresos futuros

revenue_g = []

for item in IS:
    if count < 4:
        revenue_g.append(item['revenue'])
      
        count = count + 1

revenue_g = (revenue_g[0] - revenue_g[1]) /revenue_g[1] 
print(revenue_g)

0.12770532012826136


## 9.4.2	Obtención del estado de resultados y el balance general

In [75]:
#Obtener los ingresos netos
net_income = IS[0]['netIncome']

BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={demo}').json()

# Obtenga el estado de resultados como % de los ingresos para predicciones futuras y pronostique 5 próximos años del estado de resultados

income_statement = pd.DataFrame.from_dict(IS[0],orient='index')


# Los [5:26] a continuación eliminan los elementos no necesarios que provienen de la API
income_statement = income_statement[6:33]
income_statement.columns = ['current_year']

income_statement['as_%_of_revenue'] = income_statement['current_year']/income_statement['current_year'].iloc[0]


#Pronosticar los próximos 5 años del estado de resultados
income_statement['next_year'] =  (income_statement['current_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
income_statement['next_2_year'] =  (income_statement['next_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
income_statement['next_3_year'] =  (income_statement['next_2_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
income_statement['next_4_year'] =  (income_statement['next_3_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
income_statement['next_5_year'] =  (income_statement['next_4_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 

# Obtener el balance general como porcentaje de los ingresos

balance_sheet = pd.DataFrame.from_dict(BS[0],orient='index')

balance_sheet = balance_sheet[6:-2]   #CAMBIE DE [6:]
balance_sheet.columns = ['current_year']
balance_sheet['as_%_of_revenue'] = balance_sheet['current_year'] / income_statement['current_year'].iloc[0]

#Pronosticar los próximos 5 años del balance general

balance_sheet['next_year'] =  income_statement['next_year'] ['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['next_2_year'] =  income_statement['next_2_year'] ['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['next_3_year'] =  income_statement['next_3_year']['revenue'] * balance_sheet['as_%_of_revenue'] 
balance_sheet['next_4_year'] =  income_statement['next_4_year']['revenue']  * balance_sheet['as_%_of_revenue'] 
balance_sheet['next_5_year'] =  income_statement['next_5_year']['revenue'] * balance_sheet['as_%_of_revenue']


## 9.4.3	Proyección de los flujos de caja futuros de las operaciones

In [76]:
CF_forecast = {}
CF_forecast['next_year'] = {}
CF_forecast['next_year']['netIncome'] = income_statement['next_year']['netIncome']

In [77]:

CF_forecast['next_year']['inc_depreciation'] = income_statement['next_year']['depreciationAndAmortization'] - income_statement['current_year']['depreciationAndAmortization']
CF_forecast['next_year']['inc_receivables'] = balance_sheet['next_year']['netReceivables'] - balance_sheet['current_year']['netReceivables']
CF_forecast['next_year']['inc_inventory'] = balance_sheet['next_year']['inventory'] - balance_sheet['current_year']['inventory']
CF_forecast['next_year']['inc_payables'] = balance_sheet['next_year']['accountPayables'] - balance_sheet['current_year']['accountPayables']
CF_forecast['next_year']['CF_operations'] = CF_forecast['next_year']['netIncome'] + CF_forecast['next_year']['inc_depreciation'] + (CF_forecast['next_year']['inc_receivables'] * -1) + (CF_forecast['next_year']['inc_inventory'] *-1) + CF_forecast['next_year']['inc_payables']
CF_forecast['next_year']['CAPEX'] = balance_sheet['next_year']['propertyPlantEquipmentNet'] - balance_sheet['current_year']['propertyPlantEquipmentNet'] + income_statement['next_year']['depreciationAndAmortization']

CF_forecast['next_year']['FCF'] = CF_forecast['next_year']['CAPEX'] + CF_forecast['next_year']['CF_operations']


CF_forecast['next_2_year'] = {}
CF_forecast['next_2_year']['netIncome'] = income_statement['next_2_year']['netIncome']

CF_forecast['next_2_year']['inc_depreciation'] = income_statement['next_2_year']['depreciationAndAmortization'] - income_statement['next_year']['depreciationAndAmortization']
CF_forecast['next_2_year']['inc_receivables'] = balance_sheet['next_2_year']['netReceivables'] - balance_sheet['next_year']['netReceivables']
CF_forecast['next_2_year']['inc_inventory'] = balance_sheet['next_2_year']['inventory'] - balance_sheet['next_year']['inventory']
CF_forecast['next_2_year']['inc_payables'] = balance_sheet['next_2_year']['accountPayables'] - balance_sheet['next_year']['accountPayables']
CF_forecast['next_2_year']['CF_operations'] = CF_forecast['next_2_year']['netIncome'] + CF_forecast['next_2_year']['inc_depreciation'] + (CF_forecast['next_2_year']['inc_receivables'] * -1) + (CF_forecast['next_2_year']['inc_inventory'] *-1) + CF_forecast['next_2_year']['inc_payables']
CF_forecast['next_2_year']['CAPEX'] = balance_sheet['next_2_year']['propertyPlantEquipmentNet'] - balance_sheet['next_year']['propertyPlantEquipmentNet'] + income_statement['next_2_year']['depreciationAndAmortization']
CF_forecast['next_2_year']['FCF'] = CF_forecast['next_2_year']['CAPEX'] + CF_forecast['next_2_year']['CF_operations']

CF_forecast['next_3_year'] = {}
CF_forecast['next_3_year']['netIncome'] = income_statement['next_3_year']['netIncome']

CF_forecast['next_3_year']['inc_depreciation'] = income_statement['next_3_year']['depreciationAndAmortization'] - income_statement['next_2_year']['depreciationAndAmortization']
CF_forecast['next_3_year']['inc_receivables'] = balance_sheet['next_3_year']['netReceivables'] - balance_sheet['next_2_year']['netReceivables']
CF_forecast['next_3_year']['inc_inventory'] = balance_sheet['next_3_year']['inventory'] - balance_sheet['next_2_year']['inventory']
CF_forecast['next_3_year']['inc_payables'] = balance_sheet['next_3_year']['accountPayables'] - balance_sheet['next_2_year']['accountPayables']
CF_forecast['next_3_year']['CF_operations'] = CF_forecast['next_3_year']['netIncome'] + CF_forecast['next_3_year']['inc_depreciation'] + (CF_forecast['next_3_year']['inc_receivables'] * -1) + (CF_forecast['next_3_year']['inc_inventory'] *-1) + CF_forecast['next_3_year']['inc_payables']
CF_forecast['next_3_year']['CAPEX'] = balance_sheet['next_3_year']['propertyPlantEquipmentNet'] - balance_sheet['next_2_year']['propertyPlantEquipmentNet'] + income_statement['next_3_year']['depreciationAndAmortization']
CF_forecast['next_3_year']['FCF'] = CF_forecast['next_3_year']['CAPEX'] + CF_forecast['next_3_year']['CF_operations']


CF_forecast['next_4_year'] = {}
CF_forecast['next_4_year']['netIncome'] = income_statement['next_4_year']['netIncome']

CF_forecast['next_4_year']['inc_depreciation'] = income_statement['next_4_year']['depreciationAndAmortization'] - income_statement['next_3_year']['depreciationAndAmortization']
CF_forecast['next_4_year']['inc_receivables'] = balance_sheet['next_4_year']['netReceivables'] - balance_sheet['next_3_year']['netReceivables']
CF_forecast['next_4_year']['inc_inventory'] = balance_sheet['next_4_year']['inventory'] - balance_sheet['next_3_year']['inventory']
CF_forecast['next_4_year']['inc_payables'] = balance_sheet['next_4_year']['accountPayables'] - balance_sheet['next_3_year']['accountPayables']
CF_forecast['next_4_year']['CF_operations'] = CF_forecast['next_4_year']['netIncome'] + CF_forecast['next_4_year']['inc_depreciation'] + (CF_forecast['next_4_year']['inc_receivables'] * -1) + (CF_forecast['next_4_year']['inc_inventory'] *-1) + CF_forecast['next_4_year']['inc_payables']
CF_forecast['next_4_year']['CAPEX'] = balance_sheet['next_4_year']['propertyPlantEquipmentNet'] - balance_sheet['next_3_year']['propertyPlantEquipmentNet'] + income_statement['next_4_year']['depreciationAndAmortization']
CF_forecast['next_4_year']['FCF'] = CF_forecast['next_4_year']['CAPEX'] + CF_forecast['next_4_year']['CF_operations']

CF_forecast['next_5_year'] = {}
CF_forecast['next_5_year']['netIncome'] = income_statement['next_5_year']['netIncome']

CF_forecast['next_5_year']['inc_depreciation'] = income_statement['next_5_year']['depreciationAndAmortization'] - income_statement['next_4_year']['depreciationAndAmortization']
CF_forecast['next_5_year']['inc_receivables'] = balance_sheet['next_5_year']['netReceivables'] - balance_sheet['next_4_year']['netReceivables']
CF_forecast['next_5_year']['inc_inventory'] = balance_sheet['next_5_year']['inventory'] - balance_sheet['next_4_year']['inventory']
CF_forecast['next_5_year']['inc_payables'] = balance_sheet['next_5_year']['accountPayables'] - balance_sheet['next_4_year']['accountPayables']
CF_forecast['next_5_year']['CF_operations'] = CF_forecast['next_5_year']['netIncome'] + CF_forecast['next_5_year']['inc_depreciation'] + (CF_forecast['next_5_year']['inc_receivables'] * -1) + (CF_forecast['next_5_year']['inc_inventory'] *-1) + CF_forecast['next_5_year']['inc_payables']
CF_forecast['next_5_year']['CAPEX'] = balance_sheet['next_5_year']['propertyPlantEquipmentNet'] - balance_sheet['next_4_year']['propertyPlantEquipmentNet'] + income_statement['next_5_year']['depreciationAndAmortization']
CF_forecast['next_5_year']['FCF'] = CF_forecast['next_5_year']['CAPEX'] + CF_forecast['next_5_year']['CF_operations']


## 9.4.4	Convertir los flujos de caja libres proyectados a pandas

In [78]:
#Llevar los flujos de caja proyectados a un DataFrame de pandas
CF_forec = pd.DataFrame.from_dict(CF_forecast,orient='columns')

#Formatear el DataFrame con separadores de miles
pd.options.display.float_format = '{:,.0f}'.format

print(CF_forec)


                      next_year    next_2_year    next_3_year     next_4_year  \
netIncome        45,411,565,536 51,210,864,051 57,750,763,838  65,125,843,622   
inc_depreciation  1,749,179,770  1,972,559,332  2,224,465,653   2,508,541,752   
inc_receivables   4,007,903,767  4,519,734,401  5,096,928,529   5,747,833,418   
inc_inventory        92,969,473    104,842,169    118,231,072     133,329,809   
inc_payables        713,745,034    804,894,072    907,683,327   1,023,599,317   
CF_operations    43,773,617,100 49,363,740,885 55,667,753,218  62,776,821,463   
CAPEX            27,828,487,609 31,382,333,528 35,390,024,478  39,909,518,883   
FCF              71,602,104,710 80,746,074,414 91,057,777,696 102,686,340,346   

                     next_5_year  
netIncome         73,442,760,330  
inc_depreciation   2,828,895,879  
inc_receivables    6,481,862,325  
inc_inventory        150,356,735  
inc_payables       1,154,318,396  
CF_operations     70,793,755,545  
CAPEX             45,006,

## 9.4.5	Estimación del costo de capital

In [79]:
#EBIT= IS[0]['ebitda'] - IS[0]['depreciationAndAmortization']

In [80]:
import pandas_datareader.data as web
import datetime
import requests


demo = 'api key' #Aquí debe ingresar su clave
#Interest coverage ratio = EBIT / interest expenses

def interest_coveraga_and_RF(company):
    IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={demo}').json()
    EBIT= IS[0]['ebitda'] - IS[0]['depreciationAndAmortization'] 
    interest_expense = IS[0]['interestExpense']
    interest_coverage_ratio = EBIT / interest_expense

    #RF o tasa libre de riesgo
    start = datetime.datetime(2019, 7, 10)
    end= datetime.datetime.today().strftime('%Y-%m-%d')
    #end = datetime.datetime(2020, 7, 10)
    Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    RF = RF/100
    return [RF,interest_coverage_ratio]


In [81]:
#Costo de deuda
def cost_of_debt(company, RF,interest_coverage_ratio):
    if interest_coverage_ratio > 8.5:
        credit_spread = 0.0063
    if (interest_coverage_ratio > 6.5) & (interest_coverage_ratio <= 8.5):
        credit_spread = 0.0078
    if (interest_coverage_ratio > 5.5) & (interest_coverage_ratio <=  6.5):
        credit_spread = 0.0098
    if (interest_coverage_ratio > 4.25) & (interest_coverage_ratio <=  5.49):
        credit_spread = 0.0108
    if (interest_coverage_ratio > 3) & (interest_coverage_ratio <=  4.25):
        credit_spread = 0.0122
    if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <=  3):
        credit_spread = 0.0156
    if (interest_coverage_ratio > 2.25) & (interest_coverage_ratio <=  2.5):
        credit_spread = 0.02
    if (interest_coverage_ratio > 2) & (interest_coverage_ratio <=  2.25):
    #Rating is BB
        credit_spread = 0.0240
    if (interest_coverage_ratio > 1.75) & (interest_coverage_ratio <=  2):
    #Rating is B+
        credit_spread = 0.0351
    if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <=  1.75):
    #Rating is B
        credit_spread = 0.0421
    if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <=  1.5):
    #Rating is B-
        credit_spread = 0.0515
    if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <=  1.25):
    #Rating is CCC
        credit_spread = 0.0820
    if (interest_coverage_ratio > 0.65) & (interest_coverage_ratio <=  0.8):
    #Rating is CC
        credit_spread = 0.0864
    if (interest_coverage_ratio > 0.2) & (interest_coverage_ratio <=  0.65):
    #Rating is C
        credit_spread = 0.1134
    if interest_coverage_ratio <=  0.2:
    #Rating is D
        credit_spread = 0.1512
  
    cost_of_debt = RF + credit_spread
    return cost_of_debt


def costofequity(company):
    #RF o tasa libre de riesgo
    start = datetime.datetime(2019, 7, 10)
    end= datetime.datetime.today().strftime('%Y-%m-%d')
    #end = datetime.datetime(2020, 7, 10)

    Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    RF = RF/100

#Beta
    beta = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{company}?apikey={demo}')
    beta = beta.json()
    beta = float(beta['profile']['beta'])


  #Rentabilidad de mercado
    start = datetime.datetime(2019, 7, 10)
    end= datetime.datetime.today().strftime('%Y-%m-%d')

    SP500 = web.DataReader(['sp500'], 'fred', start, end)
      #Eliminar los valores no numéricos usando el método drop
    SP500.dropna(inplace = True)

    SP500yearlyreturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[-252])-1
    
    cost_of_equity = RF+(beta*(SP500yearlyreturn - RF))
    return cost_of_equity

#Tasa efectivo de impuestos y estructura de capital
ETR = 0.3
def wacc(company):
    
    FR = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{company}?apikey={demo}').json()
    

    ETR = FR[0]['effectiveTaxRate']
    

# 
    BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={demo}').json()

    
    Debt_to = BS[0]['totalDebt'] / (BS[0]['totalDebt'] + BS[0]['totalStockholdersEquity'])
    equity_to = BS[0]['totalStockholdersEquity'] / (BS[0]['totalDebt'] + BS[0]['totalStockholdersEquity'])

    WACC = (kd*(1-ETR)*Debt_to) + (ke*equity_to)
    return WACC

In [82]:
wacc(company)

0.35446990775717824

In [83]:
RF_and_IntCov = interest_coveraga_and_RF(company)
RF = RF_and_IntCov[0]
interest_coverage_ratio = RF_and_IntCov[1]
ke = costofequity(company)
kd = cost_of_debt(company,RF,interest_coverage_ratio)
wacc_company = wacc(company)
print('wacc of ' + company + ' is ' + str((wacc_company*100))+'%')



wacc of GOOG is 35.44699077571782%


## 9.4.6	Obtención del valor presente de los flujos de caja libre futuros

In [88]:
#Lista de los flujos de caja libre para cada año

import numpy_financial as npf

FCF_List = CF_forec.iloc[-1].values.tolist()

npv = npf.npv(wacc_company,FCF_List)


## 9.4.7	Cálculo del valor terminal

In [89]:
#Valor terminal
LTGrowth = 0.02

Terminal_value = (CF_forecast['next_5_year']['FCF'] * (1+ LTGrowth)) /(wacc_company  - LTGrowth)

Terminal_value_Discounted = Terminal_value/(1+wacc_company)**4
Terminal_value_Discounted


104923793455.33406

## 9.4.8	Calcular el precio objetivo de Google

In [90]:
target_equity_value = Terminal_value_Discounted + npv
debt = balance_sheet['current_year']['totalDebt']
target_value = target_equity_value - debt
numbre_of_shares = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{company}?apikey={demo}').json()
numbre_of_shares = numbre_of_shares[0]['numberOfShares']

target_price_per_share = target_value/numbre_of_shares
target_price_per_share

print(company + ' forecasted price per stock is ' + str(target_price_per_share) )
print('the forecast is based on the following assumptions: '+ 'revenue growth: ' + str(revenue_g) + ' Cost of Capital: ' + str(wacc_company) )
print('perpetuity growth: ' + str(LTGrowth)  )


GOOG forecasted price per stock is 495.47243748223946
the forecast is based on the following assumptions: revenue growth: 0.12770532012826136 Cost of Capital: 0.35446990775717824
perpetuity growth: 0.02
